In [1]:
# install required dependencies
%pip install kagglehub
%pip install pandas
%pip install nltk
%pip install sklearn
%pip install tensorflow
%pip install matplotlib
%pip install tf-keras

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
ERROR: Could not find a version that satisfies the requirement sklearn (from versions: none)
ERROR: No matching distribution found for sklearn
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [12]:
# import all the required dependencies
import kagglehub
import random
import numpy as np
import pandas as pd
import regex as re
import nltk
from nltk.corpus import stopwords, wordnet
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from multiprocessing import Pool, cpu_count
import pickle

nltk.download('stopwords')
nltk.download('punkt_tab')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to /usr/share/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package wordnet to /usr/share/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [3]:
# Download dataset
path = kagglehub.dataset_download("tboyle10/medicaltranscriptions")
print("Path to dataset files:", path)

dataset = pd.read_csv(path + "/mtsamples.csv")
print("Head: ", dataset.head)

Path to dataset files: /kaggle/input/medicaltranscriptions
Head:  <bound method NDFrame.head of       Unnamed: 0                                        description  \
0              0   A 23-year-old white female presents with comp...   
1              1           Consult for laparoscopic gastric bypass.   
2              2           Consult for laparoscopic gastric bypass.   
3              3                             2-D M-Mode. Doppler.     
4              4                                 2-D Echocardiogram   
...          ...                                                ...   
4994        4994   Patient having severe sinusitis about two to ...   
4995        4995   This is a 14-month-old baby boy Caucasian who...   
4996        4996   A female for a complete physical and follow u...   
4997        4997   Mother states he has been wheezing and coughing.   
4998        4998   Acute allergic reaction, etiology uncertain, ...   

                medical_specialty                  

## Data processing

- We drop every other column except transcription and medical_specialty.
- We also drop any rows with empty or null transcription or medical_specialty.

- Then we drop all the classes in the excluded specialties list below. We do this as these are general terms and don't specifically map to any specialty.
- We then merge the classes with large overlaps - e.g. Neurosurgery and neurology, Neurosurgery is a subset of neurology.

In [4]:
# Drop rows with missing values in specified columns
dataset.dropna(subset=['transcription', 'medical_specialty'], inplace=True)

# Keep only relevant columns
dataset = dataset[['transcription', 'medical_specialty']]

# # Filter medical specialties with at least 30 occurrences
specialty_counts = dataset['medical_specialty'].value_counts()
# valid_specialties = specialty_counts[specialty_counts >= 50].index
# dataset = dataset[dataset['medical_specialty'].isin(valid_specialties)]

# Strip spaces in 'medical_specialty' column
dataset['medical_specialty'] = dataset['medical_specialty'].str.strip()

# Remove specific categories
excluded_specialties = [
    'Surgery',
    'SOAP / Chart / Progress Notes',
    'Office Notes',
    'Consult - History and Phy.',
    'Emergency Room Reports',
    'Discharge Summary',
    'Pain Management',
    'General Medicine',
    'Radiology',
]

dataset = dataset[~dataset['medical_specialty'].isin(excluded_specialties)]

# Define category mapping to merge similar categories
category_mapping = {
    'Neurosurgery': 'Neurology',
    'Nephrology': 'Urology',
}

# Apply category mapping
dataset['medical_specialty'] = dataset['medical_specialty'].replace(category_mapping)

# Display counts for each category
for i, (category_name, category) in enumerate(dataset.groupby("medical_specialty")):
    print(f"Category {i}: {category_name}: {len(category)}")

Category 0: Allergy / Immunology: 7
Category 1: Autopsy: 8
Category 2: Bariatrics: 18
Category 3: Cardiovascular / Pulmonary: 371
Category 4: Chiropractic: 14
Category 5: Cosmetic / Plastic Surgery: 27
Category 6: Dentistry: 27
Category 7: Dermatology: 29
Category 8: Diets and Nutritions: 10
Category 9: ENT - Otolaryngology: 96
Category 10: Endocrinology: 19
Category 11: Gastroenterology: 224
Category 12: Hematology - Oncology: 90
Category 13: Hospice - Palliative Care: 6
Category 14: IME-QME-Work Comp etc.: 16
Category 15: Lab Medicine - Pathology: 8
Category 16: Letters: 23
Category 17: Neurology: 317
Category 18: Obstetrics / Gynecology: 155
Category 19: Ophthalmology: 83
Category 20: Orthopedic: 355
Category 21: Pediatrics - Neonatal: 70
Category 22: Physical Medicine - Rehab: 21
Category 23: Podiatry: 47
Category 24: Psychiatry / Psychology: 53
Category 25: Rheumatology: 10
Category 26: Sleep Medicine: 20
Category 27: Speech - Language: 9
Category 28: Urology: 237


In [5]:
# Helper: Get synonyms from WordNet
def get_synonyms(word):
    synonyms = set()
    for syn in wordnet.synsets(word):
        for lemma in syn.lemmas():
            synonym = lemma.name().replace('_', ' ')
            if synonym.lower() != word.lower():
                synonyms.add(synonym)
    return list(synonyms)

# Synonym Replacement with random n
def synonym_replacement(text, n=None):
    words = word_tokenize(text)
    new_words = words.copy()
    eligible_words = list(set([word for word in words if word.isalpha()]))
    random.shuffle(eligible_words)

    if n is None:
        n = random.randint(1, min(3, len(eligible_words)))

    num_replaced = 0
    for word in eligible_words:
        synonyms = get_synonyms(word)
        if synonyms:
            synonym = random.choice(synonyms)
            new_words = [synonym if w == word else w for w in new_words]
            num_replaced += 1
        if num_replaced >= n:
            break
    return ' '.join(new_words)

# Word Dropout
def word_dropout(text, dropout_prob=0.1):
    words = word_tokenize(text)
    new_words = [word for word in words if random.random() > dropout_prob]
    return ' '.join(new_words) if new_words else text

# Random Swap
def random_swap(text, n=1):
    words = word_tokenize(text)
    if len(words) < 2:
        return text
    for _ in range(n):
        idx1, idx2 = random.sample(range(len(words)), 2)
        words[idx1], words[idx2] = words[idx2], words[idx1]
    return ' '.join(words)

# Apply 1–2 random augmentations
def augment_text_randomly(text):
    aug_functions = [
        synonym_replacement,
        word_dropout,
        random_swap
    ]
    num_augs = random.randint(1, 2)
    selected_augs = random.sample(aug_functions, num_augs)
    for aug in selected_augs:
        text = aug(text)
    return text

# Parallel augmentation per class
def augment_class(label_samples_tuple):
    label, samples_needed, class_samples = label_samples_tuple
    samples = class_samples.sample(n=samples_needed, replace=True)
    texts = samples['transcription'].tolist()

    with Pool(cpu_count()) as p:
        augmented_texts = p.map(augment_text_randomly, texts)

    return pd.DataFrame({
        'transcription': augmented_texts,
        'medical_specialty': label
    })

specialty_counts = dataset['medical_specialty'].value_counts()
max_count = specialty_counts.max()

# Prepare augmentation tasks
tasks = []
for label, count in specialty_counts.items():
    if count < max_count:
        samples_needed = max_count - count
        class_samples = dataset[dataset['medical_specialty'] == label]
        tasks.append((label, samples_needed, class_samples))

# Run augmentations
augmented_dfs = [augment_class(task) for task in tasks]

# Combine datasets
augmented_df = pd.concat(augmented_dfs, ignore_index=True)
dataset = pd.concat([dataset, augmented_df], ignore_index=True)


## Data processing

- We clean the text and then tokenize it and then lemmatize all the words in it.

In [6]:
from sklearn.model_selection import train_test_split

lemmatizer = None
stop_words = None

def init_worker():
    global lemmatizer, stop_words
    lemmatizer = WordNetLemmatizer()
    stop_words = set(stopwords.words('english'))

def clean_text(text):
    text = text.strip().lower()
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text)
    tokens = word_tokenize(text)
    tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words]
    return " ".join(tokens)

def apply_multiprocessing(series, func, workers=None):
    with Pool(processes=workers or cpu_count(), initializer=init_worker) as pool:
        results = pool.map(func, series)
    return results

# Use multiprocessing to speed it up
dataset['processed_transcription'] = apply_multiprocessing(dataset['transcription'], clean_text)

X_train, X_test, y_train, y_test = train_test_split(
    dataset['processed_transcription'], dataset['medical_specialty'], test_size=0.2, random_state=42, stratify=dataset['medical_specialty']
)

## Naive Bayes model

In [7]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, accuracy_score

pipeline = Pipeline([
    ('tfidataset', TfidfVectorizer(analyzer='word', stop_words='english',ngram_range=(1,4), max_df=0.8, use_idf=True, smooth_idf=True, max_features=2000)),
    ('clf', MultinomialNB())  # Train a Naive Bayes classifier
])

# Train the model
pipeline.fit(X_train, y_train)

# Make predictions
y_pred = pipeline.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy:.4f}')
print(classification_report(y_test, y_pred))

Accuracy: 0.8467
                            precision    recall  f1-score   support

      Allergy / Immunology       0.88      1.00      0.94        74
                   Autopsy       1.00      0.97      0.99        74
                Bariatrics       0.96      0.91      0.93        74
Cardiovascular / Pulmonary       0.85      0.76      0.80        74
              Chiropractic       0.69      0.91      0.78        74
Cosmetic / Plastic Surgery       0.53      0.95      0.68        74
                 Dentistry       0.92      0.95      0.93        75
               Dermatology       0.97      0.89      0.93        75
      Diets and Nutritions       0.96      1.00      0.98        74
      ENT - Otolaryngology       0.87      0.70      0.78        74
             Endocrinology       0.94      0.96      0.95        75
          Gastroenterology       0.94      0.66      0.78        74
     Hematology - Oncology       0.79      0.70      0.74        74
 Hospice - Palliative Care    

## Logistic Regression Model

In [8]:
# Implement Logistic Regression model
from sklearn.linear_model import LogisticRegression

# Create a pipeline with TF-IDF and Logistic Regression
lr_pipeline = Pipeline([
    ('tfidf', TfidfVectorizer(analyzer='word', stop_words='english', ngram_range=(1,4),
                             max_df=0.8, use_idf=True, smooth_idf=True, max_features=2000)),
    ('clf', LogisticRegression(max_iter=1000, C=1.0, solver='liblinear', multi_class='ovr'))
])

# Train the model
lr_pipeline.fit(X_train, y_train)

# Make predictions
lr_y_pred = lr_pipeline.predict(X_test)

# Evaluate the model
lr_accuracy = accuracy_score(y_test, lr_y_pred)
print(f'Logistic Regression Accuracy: {lr_accuracy:.4f}')
print(classification_report(y_test, lr_y_pred))

Logistic Regression Accuracy: 0.9001
                            precision    recall  f1-score   support

      Allergy / Immunology       0.99      1.00      0.99        74
                   Autopsy       1.00      0.97      0.99        74
                Bariatrics       0.97      0.95      0.96        74
Cardiovascular / Pulmonary       0.80      0.88      0.84        74
              Chiropractic       0.74      0.91      0.82        74
Cosmetic / Plastic Surgery       0.80      0.93      0.86        74
                 Dentistry       0.95      0.99      0.97        75
               Dermatology       0.95      0.93      0.94        75
      Diets and Nutritions       1.00      1.00      1.00        74
      ENT - Otolaryngology       0.90      0.82      0.86        74
             Endocrinology       0.95      1.00      0.97        75
          Gastroenterology       0.95      0.76      0.84        74
     Hematology - Oncology       0.80      0.88      0.84        74
 Hospice -

In [10]:
import joblib
# Save Naive Bayes model
joblib.dump(pipeline, 'naive_bayes_model.pkl')

# Save Logistic Regression model
joblib.dump(lr_pipeline, 'logistic_regression_model.pkl')

['logistic_regression_model.pkl']

## CNN model

In [13]:
# %%
# Implement CNN model for text classification
import tensorflow as tf
from tensorflow.keras.layers import Dense, Dropout, Embedding, Conv1D, GlobalMaxPooling1D
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder
import numpy as np

# Parameters
max_features = 50000  # Max vocabulary size
maxlen = 2000  # Max sequence length
embedding_dims = 500  # Embedding dimension
batch_size = 128
epochs = 20

# Convert text to sequences
tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(X_train)
X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq = tokenizer.texts_to_sequences(X_test)

# Pad sequences to ensure uniform length
X_train_pad = pad_sequences(X_train_seq, maxlen=maxlen)
X_test_pad = pad_sequences(X_test_seq, maxlen=maxlen)

# Encode labels
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded = label_encoder.transform(y_test)

# Convert to categorical
y_train_cat = tf.keras.utils.to_categorical(y_train_encoded)
y_test_cat = tf.keras.utils.to_categorical(y_test_encoded)

# Build the CNN model
model = tf.keras.Sequential([
    Embedding(max_features, embedding_dims, input_length=maxlen),
    Dropout(0.2),
    Conv1D(128, 5, activation='relu'),
    GlobalMaxPooling1D(),
    Dense(128, activation='relu'),
    Dropout(0.2),
    Dense(y_train_cat.shape[1], activation='softmax')
])

# Compile the model
model.compile(
    loss='categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

# Print model summary
model.summary()

# Train the model
history = model.fit(
    X_train_pad,
    y_train_cat,
    batch_size=batch_size,
    epochs=epochs,
    validation_split=0.2,
    verbose=1
)

# Evaluate the model
loss, cnn_accuracy = model.evaluate(X_test_pad, y_test_cat, verbose=0)
print(f'CNN Model Accuracy: {cnn_accuracy:.4f}')

# Get predictions
y_pred_probs = model.predict(X_test_pad)
y_pred_classes = np.argmax(y_pred_probs, axis=1)
y_pred_labels = label_encoder.inverse_transform(y_pred_classes)

# Print classification report
print(classification_report(y_test, y_pred_labels))

# Compare with previous models
print(f'Naive Bayes Accuracy: {accuracy:.4f}')
print(f'Logistic Regression Accuracy: {lr_accuracy:.4f}')
print(f'CNN Accuracy: {cnn_accuracy:.4f}')

# Save the CNN model
model.save('cnn_model.h5')

# Save the tokenizer
with open('cnn_tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

# Save the label encoder
with open('cnn_label_encoder.pickle', 'wb') as handle:
    pickle.dump(label_encoder, handle, protocol=pickle.HIGHEST_PROTOCOL)

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_1 (Embedding)              │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_2 (Dropout)                  │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv1d_1 (Conv1D)                    │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_max_pooling1d_1               │ ?                           │     0 (unbuilt) │
│ (GlobalMaxPooling1D)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_3 (Dropout)                  │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

Epoch 1/20
54/54 ━━━━━━━━━━━━━━━━━━━━ 19s 257ms/step - accuracy: 0.2030 - loss: 3.1680 - val_accuracy: 0.7184 - val_loss: 1.6763
Epoch 2/20
54/54 ━━━━━━━━━━━━━━━━━━━━ 12s 217ms/step - accuracy: 0.7472 - loss: 1.2488 - val_accuracy: 0.8740 - val_loss: 0.5668
Epoch 3/20
54/54 ━━━━━━━━━━━━━━━━━━━━ 12s 221ms/step - accuracy: 0.9046 - loss: 0.4106 - val_accuracy: 0.8943 - val_loss: 0.4049
Epoch 4/20
54/54 ━━━━━━━━━━━━━━━━━━━━ 12s 218ms/step - accuracy: 0.9371 - loss: 0.2397 - val_accuracy: 0.9077 - val_loss: 0.3737
Epoch 5/20
54/54 ━━━━━━━━━━━━━━━━━━━━ 12s 214ms/step - accuracy: 0.9462 - loss: 0.1822 - val_accuracy: 0.9048 - val_loss: 0.3774
Epoch 6/20
54/54 ━━━━━━━━━━━━━━━━━━━━ 11s 212ms/step - accuracy: 0.9487 - loss: 0.1713 - val_accuracy: 0.8995 - val_loss: 0.3841
Epoch 7/20
54/54 ━━━━━━━━━━━━━━━━━━━━ 11s 212ms/step - accuracy: 0.9511 - loss: 0.1406 - val_accuracy: 0.9030 - val_loss: 0.3991
Epoch 8/20
54/54 ━━━━━━━━━━━━━━━━━━━━ 11s 212ms/step - accuracy: 0.9495 - loss: 0.1493 - val_accu

## Transformers model

In [ ]:
from sklearn.preprocessing import LabelEncoder
from transformers import TFAutoModelForSequenceClassification, AutoTokenizer
from tf_keras.optimizers.legacy import Adam
from tf_keras.losses import SparseCategoricalCrossentropy
from sklearn.metrics import classification_report
import numpy as np
from tf_keras import mixed_precision

mixed_precision.set_global_policy('mixed_float16')

# Initialize the LabelEncoder
label_encoder = LabelEncoder()

y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded = label_encoder.transform(y_test)

# Parameters
max_len = 256  # Max sequence length
batch_size = 32
epochs = 10

# Load a pre-trained BERT model and tokenizer from Hugging Face
model_name = "medicalai/ClinicalBERT"
tokenizer = AutoTokenizer.from_pretrained(model_name)
transformer_model = TFAutoModelForSequenceClassification.from_pretrained(model_name, num_labels=len(label_encoder.classes_), from_pt=True)

def encode_texts(texts):
    return tokenizer(texts.tolist(), padding=True, truncation=True, max_length=max_len, return_tensors="tf")

X_train_encoded = encode_texts(X_train)
X_test_encoded = encode_texts(X_test)

# Fine-tune the model
transformer_model.compile(optimizer=Adam(learning_rate=2e-5), loss=SparseCategoricalCrossentropy(from_logits=True), metrics=['accuracy'])

# Train the model
history_transformer = transformer_model.fit(
    X_train_encoded['input_ids'], y_train_encoded,
    batch_size=batch_size,
    epochs=epochs,
    validation_split=0.2,
    verbose=1
)

# Evaluate the model
loss, transformer_accuracy = transformer_model.evaluate(X_test_encoded['input_ids'], y_test_encoded, verbose=0)
print(f'Transformer Model Accuracy: {transformer_accuracy:.4f}')

y_pred_probs_transformer = transformer_model.predict(X_test_encoded['input_ids'])
y_pred_classes_transformer = np.argmax(y_pred_probs_transformer.logits, axis=1)
y_pred_labels_transformer = label_encoder.inverse_transform(y_pred_classes_transformer)

print(classification_report(y_test, y_pred_labels_transformer))

# Compare with previous models
print(f'Naive Bayes Accuracy: {accuracy:.4f}')
print(f'Logistic Regression Accuracy: {lr_accuracy:.4f}')
print(f'CNN Accuracy: {cnn_accuracy:.4f}')
print(f'Transformer Accuracy: {transformer_accuracy:.4f}')

# Save the transformer model
transformer_model.save_pretrained('transformer_model')
tokenizer.save_pretrained('transformer_model')

# Save the label encoder
with open('transformer_label_encoder.pickle', 'wb') as handle:
    pickle.dump(label_encoder, handle, protocol=pickle.HIGHEST_PROTOCOL)

tokenizer_config.json:   0%|          | 0.00/62.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/466 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/542M [00:00<?, ?B/s]

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDistilBertForSequenceClassification: ['vocab_transform.bias', 'vocab_layer_norm.weight', 'vocab_transform.weight', 'vocab_layer_norm.bias', 'vocab_projector.weight', 'vocab_projector.bias']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFDistilBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classifier.weight', 'cla

Epoch 1/10
216/216 [==============================] - 116s 452ms/step - loss: 2.5216 - accuracy: 0.4815 - val_loss: 1.3186 - val_accuracy: 0.8107
Epoch 2/10
216/216 [==============================] - 95s 441ms/step - loss: 0.8140 - accuracy: 0.8776 - val_loss: 0.4537 - val_accuracy: 0.9088
Epoch 3/10
216/216 [==============================] - 95s 440ms/step - loss: 0.3874 - accuracy: 0.9139 - val_loss: 0.3404 - val_accuracy: 0.9106
Epoch 4/10
216/216 [==============================] - 95s 440ms/step - loss: 0.2782 - accuracy: 0.9264 - val_loss: 0.3340 - val_accuracy: 0.9146
Epoch 5/10
216/216 [==============================] - 95s 440ms/step - loss: 0.2359 - accuracy: 0.9294 - val_loss: 0.3089 - val_accuracy: 0.9117
Epoch 6/10
216/216 [==============================] - 95s 440ms/step - loss: 0.2052 - accuracy: 0.9368 - val_loss: 0.3198 - val_accuracy: 0.9077
Epoch 7/10
216/216 [==============================] - 95s 440ms/step - loss: 0.1840 - accuracy: 0.9377 - val_loss: 0.3057 - val_a